In [2]:
from datasets import load_dataset


/home/vinhpx/miniconda3/envs/llm-fp8/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

dataset = load_dataset('nvidia/OpenMathInstruct-2', split='train_1M',)

In [36]:

dataset

Dataset({
    features: ['problem', 'generated_solution', 'expected_answer', 'problem_source'],
    num_rows: 1000000
})

In [37]:
dataset[0]

{'problem': 'Solve for $y$:\n\n$$\\frac{y^2 - 3y + 2}{y - 2} = y + 1$$',
 'generated_solution': 'Start by multiplying both sides by $y - 2$ to eliminate the denominator:\n\\[ (y^2 - 3y + 2) = (y + 1)(y - 2) \\]\n\nExpand both sides:\n\\[ y^2 - 3y + 2 = y^2 - y - 2 \\]\n\nSubtract $y^2$ from both sides to get:\n\\[ -3y + 2 = -y - 2 \\]\n\nAdd $3y$ to both sides:\n\\[ 2 = 2y - 2 \\]\n\nAdd $2$ to both sides:\n\\[ 4 = 2y \\]\n\nDivide by $2$ to solve for $y$:\n\\[ y = \\frac{4}{2} \\]\n\n\\[ y = \\boxed{2} \\]',
 'expected_answer': '2',
 'problem_source': 'augmented_math'}

In [42]:
from transformers import AutoTokenizer
qwen2_5b_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Math-7B-Instruct",)

In [43]:
# {'problem': 'Solve for $y$:\n\n$$\\frac{y^2 - 3y + 2}{y - 2} = y + 1$$',
#  'generated_solution': 'Start by multiplying both sides by $y - 2$ to eliminate the denominator:\n\\[ (y^2 - 3y + 2) = (y + 1)(y - 2) \\]\n\nExpand both sides:\n\\[ y^2 - 3y + 2 = y^2 - y - 2 \\]\n\nSubtract $y^2$ from both sides to get:\n\\[ -3y + 2 = -y - 2 \\]\n\nAdd $3y$ to both sides:\n\\[ 2 = 2y - 2 \\]\n\nAdd $2$ to both sides:\n\\[ 4 = 2y \\]\n\nDivide by $2$ to solve for $y$:\n\\[ y = \\frac{4}{2} \\]\n\n\\[ y = \\boxed{2} \\]',
#  'expected_answer': '2',
#  'problem_source': 'augmented_math'}
def create_texts(examples):
    texts = []
    for prob, sol in zip(examples["problem"], examples["generated_solution"]):
        text = qwen2_5b_tokenizer.apply_chat_template(
            conversation=[
                {"role": "system", "content": "You are a helpful assistant that helps people find information."},
                {"role": "user", "content": prob},
                {"role": "assistant", "content": sol},
            ],
            tokenize=False
        )
        texts.append(text)
    return {"text": texts}

In [ ]:
dataset = dataset.map(create_texts, batched=True, remove_columns=dataset.column_names, num_proc=12)

In [45]:
dataset[0]

{'text': '<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nSolve for $y$:\n\n$$\\frac{y^2 - 3y + 2}{y - 2} = y + 1$$<|im_end|>\n<|im_start|>assistant\nStart by multiplying both sides by $y - 2$ to eliminate the denominator:\n\\[ (y^2 - 3y + 2) = (y + 1)(y - 2) \\]\n\nExpand both sides:\n\\[ y^2 - 3y + 2 = y^2 - y - 2 \\]\n\nSubtract $y^2$ from both sides to get:\n\\[ -3y + 2 = -y - 2 \\]\n\nAdd $3y$ to both sides:\n\\[ 2 = 2y - 2 \\]\n\nAdd $2$ to both sides:\n\\[ 4 = 2y \\]\n\nDivide by $2$ to solve for $y$:\n\\[ y = \\frac{4}{2} \\]\n\n\\[ y = \\boxed{2} \\]<|im_end|>\n'}